In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
raw_data = pd.read_csv('epi_r.csv')
list(raw_data.columns)

['title',
 'rating',
 'calories',
 'protein',
 'fat',
 'sodium',
 '#cakeweek',
 '#wasteless',
 '22-minute meals',
 '3-ingredient recipes',
 '30 days of groceries',
 'advance prep required',
 'alabama',
 'alaska',
 'alcoholic',
 'almond',
 'amaretto',
 'anchovy',
 'anise',
 'anniversary',
 'anthony bourdain',
 'aperitif',
 'appetizer',
 'apple',
 'apple juice',
 'apricot',
 'arizona',
 'artichoke',
 'arugula',
 'asian pear',
 'asparagus',
 'aspen',
 'atlanta',
 'australia',
 'avocado',
 'back to school',
 'backyard bbq',
 'bacon',
 'bake',
 'banana',
 'barley',
 'basil',
 'bass',
 'bastille day',
 'bean',
 'beef',
 'beef rib',
 'beef shank',
 'beef tenderloin',
 'beer',
 'beet',
 'bell pepper',
 'berry',
 'beverly hills',
 'birthday',
 'biscuit',
 'bitters',
 'blackberry',
 'blender',
 'blue cheese',
 'blueberry',
 'boil',
 'bok choy',
 'bon appétit',
 'bon app��tit',
 'boston',
 'bourbon',
 'braise',
 'bran',
 'brandy',
 'bread',
 'breadcrumbs',
 'breakfast',
 'brie',
 'brine',
 'brisk

In [3]:
raw_data.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
len(list(raw_data.columns))

680

     Transform regression into binary classifier problem
         change rating into Good/Bad 
             with Good >= 3.5
             and  Bad  <  3.5
* Use SVC instead of SVR

In [4]:
# Want to transfrom this in the a binary classification problem
## with 'rating' as the targeted variable
## Above 4 => Good Rating
## Below 4 => Bad Rating
raw_data['rating'] = raw_data['rating'].apply(lambda x: 1 if x >= 4 else 0)

raw_data.rating.describe()

count    20052.000000
mean         0.535508
std          0.498750
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: rating, dtype: float64

    A little odd since the 25th percentile is higher than the mean. It seems like most of these recipes have good ratings.

In [190]:
# Count nulls 
null_count = raw_data.isnull().sum()
null_count[null_count>0]

calories    4117
protein     4162
fat         4183
sodium      4119
dtype: int64

In [191]:
# Determine the numbers of float and object type for the features
print('Total:', len(list(raw_data.columns)))
print('Float types:', sum(raw_data.dtypes == float))
print('Object types:', sum(raw_data.dtypes == object))

Total: 680
Float types: 678
Object types: 1


In [7]:
# Get rid of Object types for calculation
temp = raw_data.drop(['title'], axis = 1)

# For features with less than 0.001 variance, that means data are pratically the same hence not very useful
## to store the features with low variance that will be drop
low_var = []

# set a threshold for low variance
thres_var = 0

for col in list(temp.columns):
    if temp[col].var() == thres_var:
        low_var.append(col)

In [10]:
# Want to find features with low correlation to the targeted variable
temp_cor = raw_data.drop(['title', 'calories', 'protein', 'fat', 'sodium'] + low_var, 1)
corrmat = temp_cor.corr()

# Create a heatmap to show correlations
# sns.heatmap(corrmat, vmax = 0.8, square = True)
# plt.show()

    Although not clear, but we can see that majority of the features are not highly correlated with each other or the targeted variable

In [11]:
# to store the features with low correlation that will be drop
low_corr = []

# Set a threshold for correlation
thres_corr = 0.025

for fea in list(corrmat.index):
    if abs(corrmat[fea]['rating']) < thres_corr:
        low_corr.append(fea)

In [14]:
from sklearn.svm import SVC
svc = SVC()
X = raw_data.drop(['rating', 'title', 'calories', 'protein', 'fat', 'sodium'] + low_var + low_corr, 1)
Y = raw_data.rating

In [15]:
len(list(X))

80

In [16]:
svc.fit(X,Y)
svc.score(X,Y)

0.59275882704967087

# Use kernel trick to cut down the numbers of features
    Radial Basis Function Kernel

In [ ]:
# The number of class labels must be greater than one. same as the cross val above
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier    #Linear classifiers (SVM, logistic regression, a.o.) with SGD training.

rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features = rbf_feature.fit_transform(X)     # look into this line here
clf = SGDClassifier()   
clf.fit(X_features, Y)
clf.score(X_features, Y)

    # Previous score from SVR
svr.score(X, Y)
> 0.038565706512988962

    # Previous cross val score
from sklearn.model_selection import cross_val_score
cross_val_score(svr, X, Y, cv=5)
> ([ 0.01818511,  0.026411  ,  0.03037705,  0.01957949,  0.02451197])

# Can use this method to fix the error above

    stratified sampling

In [180]:
# Cannot run this because unique class is only 1
from sklearn.model_selection import cross_val_score
cross_val_score(svc, X, Y, cv=5)

ValueError: The number of classes has to be greater than one; got 1